## Script to 
1. pre-process the raw dose-response files into standard format; 
2. Merge them for all pairs and common pairs; 
3. save the GDSC dosage range for all pairs and drugs

*** Just use CCLE_dose_response.csv and GDSC1_dose_response.csv in data_2019_12_05 fodler as it is gnerated from new metadata file whihc is updated (has 5 dosage drugs and new standard names fro gdsc drugs eg: 681640)

In [1]:
__author__ = 'Aanchal'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import numpy as np
import FUNCTIONS as f


In [4]:
dataname='GDSC'#'CCLE'#'CCLE'

In [5]:
if (dataname=="GDSC"):
    cl_colname_2b_mapped=dataname+' id'
    dr_colname_2b_mapped=dataname+' id (1)'
    dr_colname_2b_mapped_2=dataname+' id (2)'
    
    drug_col_name='DRUG_ID'#'Compound' #
    cl_col_name='COSMIC_ID'#'CCLE Cell Line Name' #
    dosage_col_name='dose'
    response_col_name='response'
    
    
else:
    cl_colname_2b_mapped=dataname+' name'
    dr_colname_2b_mapped=dataname+' name'
    
    drug_col_name='Compound' #
    cl_col_name='CCLE Cell Line Name' #
    dosage_col_name='Doses (uM)'
    response_col_name='Activity Data (median)'
    

#### read metadata files and raw files

In [6]:
metadata_filename="../data/metadata_2019_08_15_aanchal.xlsx"
cell_line_metadata =pd.read_excel(metadata_filename,sheetname="cell line")
drug_metadata =pd.read_excel(metadata_filename,sheetname="drug")

raw=pd.read_csv("../data/drug_response/"+dataname+"/"+dataname+"_dose_response_scores.tsv", sep="\t")
raw=raw[[cl_col_name,drug_col_name,dosage_col_name,response_col_name]] #224510 pairs, 265 drugs  #len(set(raw[drug_col_name]))

In [8]:
if(dataname=="GDSC"):
    raw=f.remove5dosageDrugs(raw,cl_col_name,drug_col_name,dosage_col_name,response_col_name) #190921 pairs left in GDSC #earlier 265 drugs, now 226 drugs

### 1. Convert to standard format
#### a) map dataset specific names to standard names

In [7]:
[dic_cl,  dic_dr]=f.getDictionary(dataname)

In [8]:
l1=raw[cl_col_name]
l2=raw[drug_col_name]

la=[]
lb=[]

for i in range(len(l1)):
    cl_name_data_specific=l1[i]
    dr_name_data_specific=l2[i]
    
    std_name_cl = dic_cl.get ( str(cl_name_data_specific) )
    std_name_dr = dic_dr.get ( dr_name_data_specific )
    
    la.append(std_name_cl)
    lb.append( std_name_dr)
       
# add these lists as cols to data frame
raw_new=raw.assign(Standard_cell_line_name = la,Standard_drug_name=lb)

#reorder cols
raw_new=raw_new[['Standard_cell_line_name',cl_col_name,'Standard_drug_name',drug_col_name,dosage_col_name,response_col_name]]
###raw_new.to_csv('../data/'+dataname+'_dose_response_no5dosageDrugs.csv',index=False)

#### b) make separate row for every dosage

In [9]:
lens = raw_new[dosage_col_name].str.split(',').map(len)

raw_final=pd.DataFrame({'Standard_cell_line_name': np.repeat(raw_new['Standard_cell_line_name'], lens),
                    cl_col_name: np.repeat(raw_new[cl_col_name], lens),
                    'Standard_drug_name': np.repeat(raw_new['Standard_drug_name'], lens),
                    drug_col_name: np.repeat(raw_new[drug_col_name], lens),
                    dosage_col_name: f.chainer(raw_new[dosage_col_name]),
                    response_col_name: f.chainer(raw_new[response_col_name])})
#rename cols
raw_final.columns = ['Standard cell line name','cell line name','Standard drug name','drug name', 'Doses (uM)','Response' ]

In [10]:
raw_final.to_csv('../data/'+dataname+'_dose_response.csv',index=False)

### Run the above code for both CCLE and GDSC befoe running the below code

### 2. Merge datasets
#### a) All pairs from both datasets

In [11]:
datasets_list=['CCLE','GDSC']
l=[]
for i in range( len (datasets_list) ):
    dataset=pd.read_csv("../data/"+datasets_list[i]+"_dose_response.csv")
    dataset=dataset[['Standard cell line name','Standard drug name','Doses (uM)','Response']]
    l.append(dataset)
data_merged=pd.concat(l) # this is the full merged dataset with all pairs (may not be in common set)

In [12]:
data_merged.to_csv('../data/MERGED_dose_response.csv')
    

#### b) Merge datasets only for common pairs 

In [13]:
import FUNCTIONS as f
datasets_list=['CCLE','GDSC']
rawFileDir="../data/"
[Intersection, Intersection_names] = f.getCommonPairs(datasets_list, rawFileDir)

0
1


In [14]:
Intersection.to_csv('../data/CCLE_GDSC1_dose_response_commonPairs.csv', index=False)

### 3. Save dosage ranges 
##### a)Get min and max dosage tested for each pair (since come CLs dont follow the same range  in a drug in GDSC
eg: drugs 1047 (Nutlin-3a), 1054 (palbo), 1060(PD), 1062 (Selu)

In [15]:
data_raw=pd.read_csv('../data/GDSC_dose_response.csv')

In [16]:
unique_drugs= data_raw['Standard drug name'].unique() 
len(unique_drugs)
drnames=[]
clnames=[]
min_dosages=[]
max_dosages=[]
nod=[]
i=0
for drname in unique_drugs:
    
    i=i+1
    df = data_raw[ data_raw['Standard drug name']==drname] 
    
    unique_cls= df['Standard cell line name'].unique() 
    print(i, drname,len(unique_cls))
    
    for clname in unique_cls:
        df2 = df[ df['Standard cell line name']==clname] 
    
        drnames.append(drname)
        clnames.append(clname)
        min_dosages.append(df2['Doses (uM)'].min())
        max_dosages.append(df2['Doses (uM)'].max())
        nod.append(len(df2['Doses (uM)'].unique()))
        

1 Tanespimycin 889
2 VX-702 890
3 Motesanib 887
4 KU-55933 887
5 Elesclomol 892
6 Afatinib 892
7 Vismodegib 893
8 PLX-4720 887
9 BX796 888
10 NU7441 885
11 SL0101 872
12 Doramapimod 877
13 JNK Inhibitor VIII 886
14 681640 797
15 Nutlin-3a 888
16 PD173074 888
17 ZM447439 836
18 RO-3306 888
19 MK-2206 864
20 Palbociclib 864
21 Dactolisib 881
22 Pictilisib 880
23 AZD8055 880
24 PD0325901 880
25 SB590885 868
26 Selumetinib 862
27 AZD6482 1044
28 CCT007093 962
29 EHT-1864 964
30 Avagacestat 1051
31 PF-4708671 950
32 Serdemetan 961
33 TW 37 963
34 CCT-018159 939
35 Rucaparib 964
36 Doxorubicin 931
37 Etoposide 940
38 Gemcitabine 926
39 Mitomycin-C 932
40 Vinorelbine 941
41 NSC-87877 936
42 Bicalutamide 930
43 QS11 931
44 CP466722 983
45 Midostaurin 941
46 CHIR-99021 938
47 Ponatinib 940
48 JNK-9L 940
49 PF-562271 921
50 HG6-64-1 936
51 JQ1 940
52 JQ12 934
53 DMOG 937
54 FTI-277 940
55 OSU-03012 930
56 Shikonin 937
57 AKT inhibitor VIII 1001
58 Embelin 932
59 FH535 928
60 PAC-1 919
61 IPA-3 9

In [17]:
dosage_range_perPair=pd.DataFrame()
dosage_range_perPair['Standard drug name']=drnames
dosage_range_perPair['Standard cell line name']=clnames
dosage_range_perPair['min_dosage']=min_dosages
dosage_range_perPair['max_dosage']=max_dosages
dosage_range_perPair['num dosages']=nod
dosage_range_perPair #(its length doesnt match with no of pairs in gdsc..debug y..bcz raw_new has duplicates bcz of two dryug ids for same drug!!!)

,Standard drug name,Standard cell line name,min_dosage,max_dosage,num dosages
0,Tanespimycin,22RV1,0.003906,1.0,9
1,Tanespimycin,23132-87,0.003906,1.0,9
2,Tanespimycin,42-MG-BA,0.003906,1.0,9
3,Tanespimycin,201T,0.003906,1.0,9
4,Tanespimycin,451Lu,0.003906,1.0,9
5,Tanespimycin,5637,0.003906,1.0,9
6,Tanespimycin,639-V,0.003906,1.0,9
7,Tanespimycin,647-V,0.003906,1.0,9
8,Tanespimycin,697,0.003906,1.0,9
9,Tanespimycin,8-MG-BA,0.003906,1.0,9


In [18]:
dosage_range_perPair.to_csv('../data_processed/'+dataname+'_pair_dosage_range.csv',index=False)

##### b) get min and max dosage tested for each drug (some CLs dont folow the same range); so just take doasge rangeof majority CLs

In [19]:
#dataname='CCLE' #ccle has same dosage range for all pairs so the below code would run only for gdsc
dataname='GDSC'
dosage_range_perPair=pd.read_csv('../data_processed/'+dataname+'_pair_dosage_range.csv')

In [20]:
unique_drugs= dosage_range_perPair['Standard drug name'].unique() 

In [21]:
min_dosages=[]
max_dosages=[]
nod=[]
for drname in unique_drugs:
    df = dosage_range_perPair[ dosage_range_perPair['Standard drug name']==drname]
    
    min_dosages.append(df['min_dosage'].value_counts().argmax())
    max_dosages.append(df['max_dosage'].value_counts().argmax())
    nod.append(df['num dosages'].mean())

In [22]:
dosage_range_perDrug=pd.DataFrame()
dosage_range_perDrug['Standard drug name']=unique_drugs
dosage_range_perDrug['min_dosage']=min_dosages
dosage_range_perDrug['max_dosage']=max_dosages
dosage_range_perDrug['num dosages']=nod
dosage_range_perDrug

,Standard drug name,min_dosage,max_dosage,num dosages
0,Tanespimycin,0.003906,1.000,9.000000
1,VX-702,0.007812,2.000,9.000000
2,Motesanib,0.007812,2.000,9.000000
3,KU-55933,0.039062,10.000,9.000000
4,Elesclomol,0.000781,0.200,9.000000
5,Afatinib,0.001953,0.500,9.000000
6,Vismodegib,0.039062,10.000,9.000000
7,PLX-4720,0.039062,10.000,9.000000
8,BX796,0.019531,5.000,9.000000
9,NU7441,0.007812,2.000,9.000000


In [23]:
dosage_range_perDrug.to_csv('../data_processed/'+dataname+'_drug_dosage_range.csv',index=False)